# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
df = pd.read_csv("./data/AviationData.csv", encoding="latin-1", low_memory=False)

df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [3]:
air_craft_se = df["Aircraft.Category"]
print(f"Empty Values: {air_craft_se.isna().sum()}")

air_craft_se.value_counts()

Empty Values: 56602


Aircraft.Category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [4]:
air_craft_se.fillna("Airplane", inplace=True)

air_craft_se.value_counts()

Aircraft.Category
Airplane             84219
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [5]:
airplane_df = df[air_craft_se == "Airplane"]
airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84219 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                84219 non-null  object 
 1   Investigation.Type      84219 non-null  object 
 2   Accident.Number         84219 non-null  object 
 3   Event.Date              84219 non-null  object 
 4   Location                84169 non-null  object 
 5   Country                 83998 non-null  object 
 6   Latitude                30507 non-null  object 
 7   Longitude               30499 non-null  object 
 8   Airport.Code            48307 non-null  object 
 9   Airport.Name            50802 non-null  object 
 10  Injury.Severity         83289 non-null  object 
 11  Aircraft.damage         81201 non-null  object 
 12  Aircraft.Category       84219 non-null  object 
 13  Registration.Number     82888 non-null  object 
 14  Make                    84159 non-null  obj

In [6]:
airplane_df["Amateur.Built"].value_counts()

Amateur.Built
No     76008
Yes     8111
Name: count, dtype: int64

In [7]:
airplane_df = airplane_df[airplane_df["Amateur.Built"] == "No"]
airplane_df["Amateur.Built"].value_counts()

Amateur.Built
No    76008
Name: count, dtype: int64

In [8]:
airplane_df = airplane_df.dropna(subset=["Publication.Date"])
airplane_df["Report.Date"] = pd.to_datetime(airplane_df["Publication.Date"], dayfirst=True)
airplane_df = airplane_df.sort_values(by=["Report.Date"])
airplane_df = airplane_df[airplane_df["Report.Date"] > "1983"]

airplane_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63340 entries, 20 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                63340 non-null  object        
 1   Investigation.Type      63340 non-null  object        
 2   Accident.Number         63340 non-null  object        
 3   Event.Date              63340 non-null  object        
 4   Location                63303 non-null  object        
 5   Country                 63181 non-null  object        
 6   Latitude                26131 non-null  object        
 7   Longitude               26126 non-null  object        
 8   Airport.Code            37023 non-null  object        
 9   Airport.Name            38508 non-null  object        
 10  Injury.Severity         62659 non-null  object        
 11  Aircraft.damage         60901 non-null  object        
 12  Aircraft.Category       63340 non-null  object    

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [9]:
filter_col = airplane_df.columns.str.contains("njur")
inj_cols = airplane_df.columns[filter_col][1::]

airplane_df[inj_cols].isna().sum() # NaN/injury type

Total.Fatal.Injuries       9511
Total.Serious.Injuries    10411
Total.Minor.Injuries       9909
Total.Uninjured            4554
dtype: int64

In [10]:
airplane_df[inj_cols].isna().all(axis=1).sum()

np.int64(140)

In [11]:
airplane_df = airplane_df[~airplane_df[inj_cols].isna().all(axis=1)] # drop injury NaNs from cols

airplane_df[inj_cols].isna().sum() # NaNs left

Total.Fatal.Injuries       9371
Total.Serious.Injuries    10271
Total.Minor.Injuries       9769
Total.Uninjured            4414
dtype: int64

In [12]:
airplane_df.loc[:,inj_cols] = airplane_df[inj_cols].fillna(0) # replace NaNs with zeros

airplane_df[inj_cols].isna().sum()

Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64

In [13]:
airplane_df.loc[:,"N_passenger"] = airplane_df.loc[:,inj_cols].sum(axis=1) # passenger total

airplane_df = airplane_df[airplane_df["N_passenger"] > 0]

passengers = airplane_df["N_passenger"]
serious_inj = airplane_df["Total.Serious.Injuries"]
fatal_inj = airplane_df["Total.Fatal.Injuries"]

airplane_df["ser_inj_rate"] = (fatal_inj + serious_inj)/passengers # injury rate

airplane_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report.Date,N_passenger,ser_inj_rate
20,20020917X01894,Accident,CHI82FEC08,1982-01-02,"YPSILANTI, MI",United States,NaN,NaN,YIP,WILLOW RUN,...,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,02-01-1983,1983-01-02,1.0,0.0
17,20020917X01656,Accident,ANC82FAG14,1982-01-02,"SKWENTA, AK",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,VMC,Unknown,Probable Cause,02-01-1983,1983-01-02,3.0,1.0
15,20020917X02117,Accident,FTW82FPG08,1982-01-02,"LITTLE ROCK, AR",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,2.0,1.0
14,20020917X02119,Accident,FTW82FPJ10,1982-01-02,"CHICKASHA, OK",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,1.0,1.0
13,20020917X02134,Accident,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,NaN,NaN,T72,HEARNE MUNICIPAL,...,0.0,0.0,0.0,IMC,Takeoff,Probable Cause,02-01-1983,1983-01-02,1.0,1.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [14]:
airplane_df["Aircraft.damage"].unique()

array(['Substantial', 'Destroyed', 'Minor', nan, 'Unknown'], dtype=object)

In [15]:
airplane_df["Aircraft.damage"].value_counts()

Aircraft.damage
Substantial    46048
Destroyed      12494
Minor           1854
Unknown           41
Name: count, dtype: int64

In [16]:
# drop unknowns and NaNs
airplane_df["Aircraft.damage"] = airplane_df["Aircraft.damage"].replace({"Unknown":np.nan})
airplane_df.dropna(subset=["Aircraft.damage"], inplace=True)
airplane_df["Aircraft.damage"].value_counts()

Aircraft.damage
Substantial    46048
Destroyed      12494
Minor           1854
Name: count, dtype: int64

In [17]:
airplane_df["is_destroyed"] = (airplane_df["Aircraft.damage"] == "Destroyed").astype(int)

airplane_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report.Date,N_passenger,ser_inj_rate,is_destroyed
20,20020917X01894,Accident,CHI82FEC08,1982-01-02,"YPSILANTI, MI",United States,NaN,NaN,YIP,WILLOW RUN,...,0.0,1.0,VMC,Takeoff,Probable Cause,02-01-1983,1983-01-02,1.0,0.0,0
17,20020917X01656,Accident,ANC82FAG14,1982-01-02,"SKWENTA, AK",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,VMC,Unknown,Probable Cause,02-01-1983,1983-01-02,3.0,1.0,1
15,20020917X02117,Accident,FTW82FPG08,1982-01-02,"LITTLE ROCK, AR",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,2.0,1.0,1
14,20020917X02119,Accident,FTW82FPJ10,1982-01-02,"CHICKASHA, OK",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02,1.0,1.0,1
13,20020917X02134,Accident,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,NaN,NaN,T72,HEARNE MUNICIPAL,...,0.0,0.0,IMC,Takeoff,Probable Cause,02-01-1983,1983-01-02,1.0,1.0,1


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [18]:
airplane_df["Make"].value_counts()

Make
Cessna                  17456
Piper                    9435
CESSNA                   4567
Beech                    3439
PIPER                    2668
                        ...  
ICP SRL                     1
MXR TECHNOLOGIES INC        1
WINGREN NORMAN              1
STORY CHARLES BRAD          1
PIPISTREL D O O             1
Name: count, Length: 1738, dtype: int64

Consolidate

In [19]:

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["CESSNA AIRCRAFT CO", "CESSNA AIRCRAFT COMPANY", "CESSNA AIRCRAFT", "Cessna"], value="CESSNA")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["PIPER AIRCRAFT INC", "PIPER AIRCRAFT CORPORATION", "PIPER AIRCRAFT", "Piper"], value="PIPER")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["BEECHCRAFT", "HAWKER BEECHCRAFT", "HAWKER BEECHCRAFT CORP", "HAWKER BEECH", "Beech"], value="BEECH") 

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["THE BOEING COMPANY", "BOEING COMPANY", "BOEING STEARMAN", "Boeing"], value="BOEING")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["MOONEY AIRCRAFT CORP.", "MOONEY AIRPLANE CO INC", "MOONEY INTERNATIONAL", "Mooney"], value="MOONEY")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["GRUMMAN ACFT ENG COR-SCHWEIZER", "GRUMMAN AMERICAN AVN. CORP", "GRUMMAN AMERICAN AVN. CORP", "Grumman-schweizer", "Schweizer", "Grumman"], value= "GRUMMAN")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AIRBUS INDUSTRIES", "Airbus Industrie", "Airbus"], value="AIRBUS")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["MAULE AIRCRAFT CORP", "Maule"], value="MAULE")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AERONCA AIRCRAFT CORPORATION", "AERONCA CHAMPION", "AERONCA CHAMP", "Aeronca", "Champion"], value="AERONCA")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AIR TRACTOR", "AIR TRACTOR INC", "Air Tractor"], value="AIR TRACTOR")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["CIRRUS", "CIRRUS DESIGN CORP", "CIRRUS DESIGN CORP.", "Cirrus"], value="CIRRUS")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["ERCOUPE", "ERCOUPE (ENG & RESEARCH CORP.)", "Ercoupe"], value="ERCOUPE")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["AVIAT AIRCRAFT INC", "AVIAT", "Aviat"], value="AVIAT")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["ROCKWELL", "ROCKWELL INTERNATIONAL", "Rockwell"], value="ROCKWELL")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["LUSCOMBE", "Luscombe"], value="LUSCOMBE")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["TAYLORCRAFT", "Taylorcraft"], value="TAYLORCRAFT")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["STINSON", "Stinson"], value="STINSON")

airplane_df["Make"] = airplane_df["Make"].replace(to_replace=["EMBRAER", "Embraer"], value="EMBRAER")

filtered_makes = airplane_df["Make"].value_counts()[airplane_df["Make"].value_counts() > 50]
filtered_makes[:25]

Make
CESSNA               22063
PIPER                12136
BEECH                 4444
GRUMMAN               1484
Bell                  1341
BOEING                1159
MOONEY                1135
AERONCA                883
AIR TRACTOR            849
Bellanca               666
Robinson               592
Hughes                 524
MAULE                  502
STINSON                369
LUSCOMBE               349
CIRRUS                 346
ROCKWELL               320
TAYLORCRAFT            316
Mcdonnell Douglas      314
De Havilland           293
Aero Commander         273
North American         257
AVIAT                  209
Aerospatiale           206
Hiller                 187
Name: count, dtype: int64

Filter by Makes

In [20]:
airplane_df = airplane_df[airplane_df["Make"].isin(filtered_makes.index)].copy()

airplane_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 54666 entries, 20 to 88784
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                54666 non-null  object        
 1   Investigation.Type      54666 non-null  object        
 2   Accident.Number         54666 non-null  object        
 3   Event.Date              54666 non-null  object        
 4   Location                54639 non-null  object        
 5   Country                 54544 non-null  object        
 6   Latitude                21632 non-null  object        
 7   Longitude               21629 non-null  object        
 8   Airport.Code            32530 non-null  object        
 9   Airport.Name            33933 non-null  object        
 10  Injury.Severity         54666 non-null  object        
 11  Aircraft.damage         54666 non-null  object        
 12  Aircraft.Category       54666 non-null  object    

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [21]:
airplane_df.dropna(subset=["Model"], inplace=True)

airplane_df.groupby(["Model", "Make"]).count().loc["100"]

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report.Date,N_passenger,ser_inj_rate,is_destroyed
Make,,,,,,,,,,,,,,,,,,,,,
AERO COMMANDER,7,7,7,7,7,7,7,7,3,5,...,7,7,7,0,5,7,7,7,7,7
Aero Commander,26,26,26,26,26,26,9,9,13,17,...,26,26,26,25,26,26,26,26,26,26
BEECH,10,10,10,10,10,10,3,3,7,7,...,10,10,10,7,9,10,10,10,10,10
ROCKWELL,1,1,1,1,1,1,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [22]:
airplane_df.loc[:, "make_model"] = airplane_df["Make"] + "_" + airplane_df["Model"].str.upper()

airplane_df["make_model"].value_counts()

make_model
CESSNA_152                                1806
CESSNA_172                                1469
CESSNA_172N                                976
PIPER_PA-28-140                            741
CESSNA_172M                                675
                                          ... 
RAYTHEON AIRCRAFT COMPANY_HAWKER 800XP       1
DEHAVILLAND_DHC-2-L-20                       1
STINSON_JR. SR                               1
EMBRAER_EMB500                               1
AIRBUS_A320-271N                             1
Name: count, Length: 4717, dtype: int64

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

Engine.Type Cleanup

In [23]:
airplane_df["Engine.Type"].value_counts()

Engine.Type
Reciprocating      46042
Turbo Prop          2307
Turbo Shaft         1551
Turbo Fan           1049
Unknown              871
Turbo Jet            266
Geared Turbofan        1
UNK                    1
Name: count, dtype: int64

In [24]:
airplane_df["Engine.Type"] = airplane_df["Engine.Type"].replace(
    {"Unknown": np.nan, "UNK": np.nan}
)

# filter out single engine values
filtr_engine = airplane_df["Engine.Type"].value_counts()[airplane_df["Engine.Type"].value_counts() > 1] 

airplane_df = airplane_df[airplane_df["Engine.Type"].isin(filtr_engine.index)]

airplane_df["Engine.Type"].value_counts()

Engine.Type
Reciprocating    46042
Turbo Prop        2307
Turbo Shaft       1551
Turbo Fan         1049
Turbo Jet          266
Name: count, dtype: int64

In [25]:
airplane_df["Number.of.Engines"].value_counts()

Number.of.Engines
1.0    43821
2.0     6613
3.0      214
4.0      212
0.0        1
Name: count, dtype: int64

Drop Zero Engines

In [26]:
airplane_df = airplane_df[airplane_df["Number.of.Engines"] > 0.0]

airplane_df["Number.of.Engines"].value_counts()

Number.of.Engines
1.0    43821
2.0     6613
3.0      214
4.0      212
Name: count, dtype: int64

Clean Weather.Condition

In [27]:
airplane_df["Weather.Condition"].unique()


array(['VMC', 'IMC', 'UNK', nan, 'Unk'], dtype=object)

In [28]:
airplane_df["Weather.Condition"] = airplane_df["Weather.Condition"].replace(
    {"UNK": np.nan, "Unk": np.nan}
)

airplane_df["Weather.Condition"].value_counts()


Weather.Condition
VMC    46160
IMC     3916
Name: count, dtype: int64

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [29]:
airplane_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50860 entries, 20 to 88632
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                50860 non-null  object        
 1   Investigation.Type      50860 non-null  object        
 2   Accident.Number         50860 non-null  object        
 3   Event.Date              50860 non-null  object        
 4   Location                50849 non-null  object        
 5   Country                 50741 non-null  object        
 6   Latitude                19570 non-null  object        
 7   Longitude               19567 non-null  object        
 8   Airport.Code            31148 non-null  object        
 9   Airport.Name            32567 non-null  object        
 10  Injury.Severity         50860 non-null  object        
 11  Aircraft.damage         50860 non-null  object        
 12  Aircraft.Category       50860 non-null  object    

In [30]:
airplane_df = airplane_df.drop(columns=["FAR.Description", "Air.carrier", "Schedule"])

In [31]:
airplane_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50860 entries, 20 to 88632
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                50860 non-null  object        
 1   Investigation.Type      50860 non-null  object        
 2   Accident.Number         50860 non-null  object        
 3   Event.Date              50860 non-null  object        
 4   Location                50849 non-null  object        
 5   Country                 50741 non-null  object        
 6   Latitude                19570 non-null  object        
 7   Longitude               19567 non-null  object        
 8   Airport.Code            31148 non-null  object        
 9   Airport.Name            32567 non-null  object        
 10  Injury.Severity         50860 non-null  object        
 11  Aircraft.damage         50860 non-null  object        
 12  Aircraft.Category       50860 non-null  object    

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [33]:
airplane_df.to_csv("data/AviationData_cleaned.csv", index=False)